In [60]:
class TobiiObject:
    
    #Basic accesable variables
    data = {}
    headers = []
    participants = {}
    
    #Init
    def __init__(self, tsvPath):
        self.addParticipants(tsvPath)
        
    #Init method
    def _importTobiiTsv(self, tsvPath):
        #Import csv library
        import csv
        #Create a container for temporary data
        tempData = []
        tempHeaders = []
        index = 0
        #Open the .tsv file and append it to a python list
        with open(tsvPath,'rb') as tsvIn:
            tsvIn = csv.reader(tsvIn, delimiter='\t')
            for row in tsvIn:
                tempData.append(row)
        #Extract all the headers and append to a python list with the index
        for header in tempData[0]:
            tempHeaders.append(str(header))
            #print str(index) + ": " + header
            index += 1
        #Remove the headers from the data list
        tempData = tempData[1:]
        #Return the two data lists
        return [tempData, tempHeaders]
    
    # ---------- Public methods! ---------------
    def addParticipants(self, tsvPath):
        #Load the data and headers
        data, self.headers = self._importTobiiTsv(tsvPath)
        #Load the participants
        ###  self.participants = self._getParticipants()
        #Load the individual participants mediarows
        ###  self.participants = self._getMediaRowForParticipants(self._getParticipants())
        participants = self._getParticipants(data)
        self.addMediaRowForParticipants(self._getMediaRowForParticipants(participants, data))
        #Create saccade data for each participant for each media element [AmountOfSaccades, SaccadeMeanDuratio, SaccadeMeanVelocity]
        self._createSaccadeData(data, participants)
        #Create pupil dilation data for each participant for each media element [Mean pupil dilation, mean pupil variance, mean left pupil dilation, mean right pupil dilation]
        self._createPupilData(data, participants)
        #Create fixation data for each participant for each media element [FixationIndexes]
        self._createFixationData(data, participants)
        #Create mean for each image 
        self._createGeneralMeanDataContainer()
        self._createGeneralImageMeanData()
        self._getAnswers(data)
    def printHeaders(self):
        index = 0
        for header in self.headers:
            print str(index) + ": " + header
            index += 1
        
    # !!!!!!!!! PRIVATE METHODS !!!!!!!!!!!!
    
    def _getHeaderIndex(self, headerName):
        #Getting header index because header move according to content of the Tobii test (questions, AOI's, etc..)
        index = 0
        #Search through the headers and return if the header name is found
        for header in self.headers:
            if header == headerName:
                return index
            index += 1
            #If not found return "ERROR"
        return "ERROR"
    
    def _getParticipants(self, data):
        tempParticipants = {}
        #Get the index for ParticipantName
        participantNameIndex = self._getHeaderIndex("ParticipantName")
        #Go through the data and add participant names that are not already in the dictionary
        for row in data:
            if row[participantNameIndex] not in tempParticipants.keys():
                tempParticipants[row[participantNameIndex]] = {}
        return tempParticipants
        
    def _getMediaRowForParticipants(self, participantsIn, data):
        #Get indexes for MediaName and ParticipantName
        participantNameIndex = self._getHeaderIndex("ParticipantName")
        mediaNameIndex = self._getHeaderIndex("MediaName")
        #For each participant get the datarows for each media element, then afterwards get the starting row and ending row
        for participant in participantsIn:
            media = {}
            index = 0
            for row in data:
                if participant == row[participantNameIndex] and row[mediaNameIndex] != "":
                    if row[mediaNameIndex] not in media:
                        media[row[mediaNameIndex]] = [index]
                    else:
                        media[row[mediaNameIndex]].append(index)
                index += 1
            media2 = {}
            for key in media.keys():
                first = media[key][0]
                last = media[key][-1]
                media2[key] = [first, last]
            participantsIn[participant]["media"] = media2
        return participantsIn
    
    def _stringToFloat(self, string):
        for idx in xrange(len(string)):
            if string[idx] == ".":
                return float(string[:idx]) + float(string[idx:])
        return "ERROR"
    
    def addMediaRowForParticipants(self, participantsIn):
        for participant in participantsIn:
            self.participants[participant] = {}
            self.participants[participant]["media"] = participantsIn[participant]["media"]
    
    #------------- SACCADE METHODS-----------------------         
    def _calculateIndexAmount(self, saccadeList):
            return len(saccadeList)
    
    def _calculateSaccadeDurationMean(self, durationDict):
        dictSum = sum(durationDict.values())
        dictLen = len(durationDict.values())
        return float(dictSum) / float(dictLen)
    
    def _calculateSaccadeVelocityMean(self, velocityDict):
        velSum = sum(velocityDict.values())
        velLen = len(velocityDict.values())
        return float(velSum) / float(velLen)
            
    def _createSaccadeData(self, data, participants):
        #Saccade index in the headers
        sacIdx = self._getHeaderIndex("SaccadeIndex")
        recTimestampIdx = self._getHeaderIndex("RecordingTimestamp")
        sacAmpIdx = self._getHeaderIndex("SaccadicAmplitude")
        #For each participant
        for participant in participants:
            self.participants[participant]["saccadeData"] = {}
            #Fir each media element
            for image in self.participants[participant]["media"]:
                #Set the starting and ending data row indexes
                fromRow, toRow = self.participants[participant]["media"][image]
                #Variables
                #Index to hold all of the indexes in order to count the amount of saccades
                index = []
                #Duration dictionary to hold the duration of each saccade in the index
                duration = {}
                #Velocity dictionary in order to hold the velocity for each saccade in the index
                velocity = {}
                #Previous and current saccade indexes to compare eachother
                prevSacIdx = None
                currSacIdx = None
                #Saccade duration start and end in order to calculate the duration
                sacDurStart = 0
                sacDurEnd = 0
                for row in data[fromRow:toRow]:
                    #Update the previous and current saccade indexes
                    prevSacIdx = currSacIdx
                    currSacIdx = row[sacIdx]
                    # If the saccade index is not already in the list, append it
                    if row[sacIdx] not in index and row[sacIdx] != "":
                        index.append(row[sacIdx])
                    #Check if previous was empty and current is a saccadeIndex then set the start time
                    if currSacIdx != "" and currSacIdx != None and prevSacIdx == "":
                        sacDurStart = row[recTimestampIdx]
                    #Check if previous is a saccade and current is not, then set the end time
                    elif prevSacIdx != "" and currSacIdx == "":
                        sacDurEnd = row[recTimestampIdx]
                        tempDur = int(sacDurEnd) - int(sacDurStart)
                        if tempDur < 1000:
                            #Calculate the duration of the saccade
                            duration[prevSacIdx] = int(sacDurEnd) - int(sacDurStart)
                            #Cant remember it right now will write later
                            if row[sacAmpIdx] != "":
                                curAmp = str(row[sacAmpIdx]).replace(",",".")
                                curAmp = self._stringToFloat(curAmp)
                                #Get the duration and conver to to seconds
                                curDur = float(duration[prevSacIdx])#/1000 #Seconds
                                #Calculate the velocity
                                velocity[prevSacIdx] = curAmp / curDur
                            
                indexTotal = self._calculateIndexAmount(index)
                durationMean = self._calculateSaccadeDurationMean(duration)
                velocityMean = self._calculateSaccadeVelocityMean(velocity)
                self.participants[participant]["saccadeData"][image] = [indexTotal, durationMean, velocityMean]
                
    # ------------------- PUPIL DILATION ------------------------------
    def _createPupilData(self, data, participants):
        #Import numpy library
        import numpy as np
        #Index in the headers
        pupilLeftIdx = self._getHeaderIndex("PupilLeft")
        pupilRightIdx = self._getHeaderIndex("PupilRight")
        #For each participant
        for participant in participants:
            self.participants[participant]["pupilData"] = {}
            #Fir each media element
            for image in self.participants[participant]["media"]:
                #Set the starting and ending data row indexes
                fromRow, toRow = self.participants[participant]["media"][image]
                #Variables to hold the data
                pupilListLeft = []
                pupilListRight = []
                pupilListMean = []
                #Go through the data and create a list with the pupil data
                for row in data[fromRow:toRow]:
                    if row[pupilLeftIdx] != "" and row[pupilRightIdx] != "":
                        tmpPupilLeft = str(row[pupilLeftIdx]).replace(",",".")
                        tmpPupilLeft = self._stringToFloat(tmpPupilLeft)
                        pupilListLeft.append(tmpPupilLeft)
                        tmpPupilRight = str(row[pupilRightIdx]).replace(",",".")
                        tmpPupilRight = self._stringToFloat(tmpPupilRight)
                        pupilListRight.append(tmpPupilRight)
                        tmpPupilMean = (tmpPupilLeft + tmpPupilRight) / 2
                        pupilListMean.append(tmpPupilMean)
                #Create mean values for that list
                pupilLeftMean = sum(pupilListLeft)/len(pupilListLeft)
                pupilRightMean = sum(pupilListRight)/len(pupilListRight)
                pupilMeanMean = sum(pupilListMean)/len(pupilListMean)
                pupilVariance = np.var(pupilListMean)
                self.participants[participant]["pupilData"][image] = [pupilMeanMean, pupilVariance, pupilLeftMean, pupilRightMean]

    def _createFixationData(self, data, participants):
        #Get the header for the fixation index
        fixIdx = self._getHeaderIndex("FixationIndex")
        #For each participant
        for participant in participants:
            self.participants[participant]["fixationData"] = {}
            #For each media element
            for image in self.participants[participant]["media"]:
                 #Set the starting and ending data row indexes
                fromRow, toRow = self.participants[participant]["media"][image]
                #Index to hold all of the indexes in order to count the amount of fixations
                index = []
                for row in data[fromRow:toRow]:
                     # If the fixation index is not already in the list, append it
                    if row[fixIdx] not in index and row[fixIdx] != "":
                        index.append(row[fixIdx])
                indexTotal = self._calculateIndexAmount(index)
                self.participants[participant]["fixationData"][image] = [indexTotal]
                
    def _createGeneralMeanDataContainer(self):
        #This function creates all of the data containers which makes sure that no errors are made when trying to fill a dict with data
        self.data["imageLists"] = {}
        for participant in self.participants:
            for dictKey in self.participants[participant]:
                if dictKey not in self.data["imageLists"].keys() and not "media":
                    data["imageLists"][dictKey] = {}
                if dictKey is not "media":
                    for image in self.participants[participant][dictKey]:
                        if image not in self.data["imageLists"].keys():
                            self.data["imageLists"][image] = {"saccadeIndex":[], "saccadeDuration":[], "saccadeVelocity":[], "pupilMean":[], "pupilVariance":[], "fixationMean":[]}
    
    def _createGeneralImageMeanData(self):
        for participant in self.participants:
            for key in self.participants[participant].keys():
                if key == "fixationData":
                    for image in self.participants[participant][key]:
                        self.data["imageLists"][image]["fixationMean"].append(self.participants[participant][key][image][0])
                if key == "saccadeData":
                    for image in self.participants[participant][key]:
                        self.data["imageLists"][image]["saccadeIndex"].append(self.participants[participant][key][image][0])
                        self.data["imageLists"][image]["saccadeDuration"].append(self.participants[participant][key][image][1])
                        self.data["imageLists"][image]["saccadeVelocity"].append(self.participants[participant][key][image][2])
                if key == "pupilData":
                    for image in self.participants[participant][key]:
                        self.data["imageLists"][image]["pupilMean"].append(self.participants[participant][key][image][0])
                        self.data["imageLists"][image]["pupilVariance"].append(self.participants[participant][key][image][1])

    def _getAnswers(self, data):
        participant = data[-1][4]
        participantGroup = participant[-1]
        answerIndexA = [11,12,13,14,15,16,23,24,25,26,27,28,35,36,37,38,39,40,47,48,49,50,51,52]
        answerIndexB = [59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105]
        answerIndex = []
        answerKey = [True, True, True, False, False, False, False, True, True, False, False, False, True, False, True, False, True, False, True, False, True, False, True, False]
        correctAnswersList = []
        if participantGroup == "a":
            answerIndex = answerIndexA
        else:
            answerIndex = answerIndexB
        index = 0
        for answer in answerIndex:
            correct = True
            if data[-1][answer].lstrip() == "True":
                correct = True
            else: 
                correct = False
            if answerKey[index] == correct:
                correct = True
            else:
                correct = False
            correctAnswersList.append(correct)
            index += 1
        self.participants[participant]["answers"] = correctAnswersList
        
    def getAnswerData(self, participant):
        index = 0
        for answer in self.participants[participant]["answers"]:
            if answer == True:
                index += 1
        math = float(index) / float(len(self.participants[participant]["answers"])) * 100
        print str(participant) +" answered " + str(math) + "% correct."

In [48]:
print tobii.participants["p847b"]["answers"]

[True, False, True, True, True, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True, False, True, False, True]


In [62]:
tobii = TobiiObject("C:\Users\Simon\Documents\Skole\Master\participants\p665a.tsv")

In [63]:
tobii.addParticipants("C:\Users\Simon\Documents\Skole\Master\participants\p847b.tsv")

In [4]:
#Combine the global and local questions
imageData = {"M1A":{},
            "M2A":{},
            "M3A":{},
            "M4A":{},
            "M1B":{},
            "M2B":{},
            "M3B":{},
            "M4B":{}}
def combineImageQuestions(questionList, attribute):
    image = questionList[0][:3]
    imageData[image] = {}
    imageData[image][attribute] = {}
    for variable in tobii.data["imageLists"][questionList[0]]:
        imageData[image][attribute][variable] = []
        for index in xrange(len(tobii.data["imageLists"][questionList[0]][variable])):
            image0 = tobii.data["imageLists"][questionList[0]][variable][index]
            image1 = tobii.data["imageLists"][questionList[1]][variable][index]
            image2 = tobii.data["imageLists"][questionList[2]][variable][index]
            mean = image0 + image1 + image2
            mean = mean/3
            imageData[image][attribute][variable].append(mean)
        
        

In [5]:
imagePairs = [["M1A-Q01.png", "M1A-Q02.png", "M1A-Q03.png", "local"],
             ["M1A-Q04.png", "M1A-Q05.png", "M1A-Q06.png", "global"],
             ["M2A-Q07.png", "M2A-Q08.png", "M2A-Q09.png", "local"],
             ["M2A-Q10.png", "M2A-Q11.png", "M2A-Q12.png", "global"],
             ["M3A-Q13.png", "M3A-Q14.png", "M3A-Q15.png", "local"],
             ["M3A-Q16.png", "M3A-Q17.png", "M3A-Q18.png", "global"],
             ["M4A-Q19.png", "M4A-Q20.png", "M4A-Q24.png", "local"],
             ["M4A-Q21.png", "M4A-Q22.png", "M4A-Q23.png", "global"],
             ["M1B-Q01.png", "M1B-Q02.png", "M1B-Q03.png", "local"],
             ["M1B-Q04.png", "M1B-Q05.png", "M1B-Q06.png", "global"],
             ["M2B-Q07.png", "M2B-Q08.png", "M2B-Q09.png", "local"],
             ["M2B-Q10.png", "M2B-Q11.png", "M2B-Q12.png", "global"],
             ["M3B-Q13.png", "M3B-Q14.png", "M3B-Q15.png", "local"],
             ["M3B-Q16.png", "M3B-Q17.png", "M3B-Q18.png", "global"],
             ["M4B-Q19.png", "M4B-Q20.png", "M4B-Q24.png", "local"],
             ["M4B-Q21.png", "M4B-Q22.png", "M4B-Q23.png", "global"]]

for pair in imagePairs:
    questionList = pair[:3]
    attribute = pair[3]
    combineImageQuestions(questionList, attribute)

In [6]:
def statTest(image1, image2, attribute="global"):
    from scipy import stats
    data = {}
    image1 = imageData[image1][attribute]
    image2 = imageData[image2][attribute]
    for variable in image1:
        data[variable] = []
        #Test for normal distribution
        normalDist = stats.shapiro(image1[variable] + image2[variable])[1]
        pValue = 0.0
        if normalDist <= 0.05:
            print "Parametric test:"
            pValue = stats.ttest_ind(image1[variable], image2[variable])[1]
        else:
            print "Non-parametric test:"
            pValue = stats.mannwhitneyu(image1[variable], image2[variable])[1]
        image1VariableMean = sum(image1[variable]) / len(image1[variable])
        image2VariableMean = sum(image2[variable]) / len(image2[variable])
        print "\t" + str(variable) + ": " + str(pValue) + " " + str(image1VariableMean) + " " + str(image2VariableMean)
statTest("M1A", "M1B")

ValueError: Data must be at least length 3.

In [ ]:
statTest("M3A", "M3B")

In [ ]:
print tobii.data["imageLists"]["M1A-Q06.png"]["saccadeVelocity"]
print tobii.data["imageLists"]["M1B-Q06.png"]["saccadeVelocity"]

In [ ]:
print tobii.participants["p665a"]["saccadeData"]["M1A-Q01.png"]
print tobii.participants["p665a"]["saccadeData"]["M3A-Q18.png"]

In [ ]:
print imageData["M1A"]["global"].keys()

In [ ]:
print imageData["M1A"]["global"]["saccadeVelocity"]
print imageData["M1B"]["global"]["saccadeVelocity"]

In [ ]:
print imageData["M2A"]["global"]["saccadeVelocity"]
print imageData["M2B"]["global"]["saccadeVelocity"]

In [ ]:
print imageData["M3A"]["global"]["saccadeVelocity"]
print imageData["M3B"]["global"]["saccadeVelocity"]

In [ ]:
print imageData["M4A"]["global"]["saccadeVelocity"]
print imageData["M4B"]["global"]["saccadeVelocity"]

In [10]:
tobii.printHeaders()

0: ﻿ExportDate
1: StudioVersionRec
2: StudioProjectName
3: StudioTestName
4: ParticipantName
5: [M1AE-Q1]Value
6: [M1AE-Q2]Value
7: [M1AE-Q3]Value
8: [M1AE-Q4]Value
9: [M1AE-Q5]Value
10: [M1AE-Q6]Value
11: [M1A-Q01]Value
12: [M1A-Q02]Value
13: [M1A-Q03]Value
14: [M1A-Q04]Value
15: [M1A-Q05]Value
16: [M1A-Q06]Value
17: [M2AE-Q10]Value
18: [M2AE-Q11]Value
19: [M2AE-Q12]Value
20: [M2AE-Q7]Value
21: [M2AE-Q8]Value
22: [M2AE-Q9]Value
23: [M2A-Q07]Value
24: [M2A-Q08]Value
25: [M2A-Q09]Value
26: [M2A-Q10]Value
27: [M2A-Q11]Value
28: [M2A-Q12]Value
29: [M3AE-Q13]Value
30: [M3AE-Q14]Value
31: [M3AE-Q15]Value
32: [M3AE-Q16]Value
33: [M3AE-Q17]Value
34: [M3AE-Q18]Value
35: [M3A-Q13]Value
36: [M3A-Q14]Value
37: [M3A-Q15]Value
38: [M3A-Q16]Value
39: [M3A-Q17]Value
40: [M3A-Q18]Value
41: [M4AE-Q19]Value
42: [M4AE-Q20]Value
43: [M4AE-Q21]Value
44: [M4AE-Q22]Value
45: [M4AE-Q23]Value
46: [M4AE-Q24]Value
47: [M4A-Q19]Value
48: [M4A-Q20]Value
49: [M4A-Q21]Value
50: [M4A-Q22]Value
51: [M4A-Q23]Value
52: 

In [21]:
data, headers = tobii._importTobiiTsv("C:\Users\Simon\Documents\Skole\Master\participants\p847b.tsv")

In [22]:
data[-1][5]

'Extremly low mental effort'

In [37]:
def _getAnswers(data):
    participant = data[-1][4]
    participantGroup = participant[-1]
    answerIndexA = [11,12,13,14,15,16,23,24,25,26,27,28,35,36,37,38,39,40,47,48,49,50,51,52]
    answerIndexB = [59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105]
    answerIndex = []
    answerKey = [True, True, True, False, False, False, False, True, True, False, False, False, True, False, True, False, True, False, True, False, True, False, True, False]
    correctAnswersList = []
    if participantGroup == "a":
        answerIndex = answerIndexA
    else:
        answerIndex = answerIndexB
    index = 0
    for answer in answerIndex:
        correct = True
        if data[-1][answer].lstrip() == "True":
            correct = True
        else: 
            correct = False
        if answerKey[index] == correct:
            correct = True
        else:
            correct = False
        correctAnswersList.append(correct)
        index += 1
_getAnswers(data)

b


In [66]:
for participant in tobii.participants.keys():
    tobii.getAnswerData(participant)

p847b answered 79.1666666667% correct.
p665a answered 45.8333333333% correct.
